<a href="https://colab.research.google.com/github/WasudeoGurjalwar/AGENTIC_AI_TRAININGS_Rocky/blob/main/EV_Charging_Station_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Implement a multi-agent EV Charging station information system and booking support support workflow using LangGraph. The workflow should include a Classifier Agent to route queries (Charging Station, Booking , Cafe) to specialized agents (Charging Station, Booking , Cafe). The final responses from the specialized agents should be aggregated into a single answer. Use Google Gemini (gemini-2.5-flash-lite) initializing them in separate blocks after installing necessary libraries and configuring API keys.

## Install necessary libraries

### Subtask:
Install LangChain, LangGraph, and the libraries for interacting with Google Gemini and OpenAI APIs.


**Reasoning**:
Install the necessary libraries using pip.



In [2]:
%pip install --quiet langchain langgraph langchain-google-genai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 13.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.


## Configure api keys

### Subtask:
Add code to securely load and configure API keys for Google Gemini and OpenAI.


**Reasoning**:
The subtask is to securely load and configure API keys for Google Gemini and OpenAI. This involves importing `getpass`, prompting the user for the keys, and setting them as environment variables. These steps can be done in a single code block.



In [3]:
import getpass
import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

## Initialize language models

### Subtask:
Initialize LLMs: from Google (Gemini)


**Reasoning**:
Import the necessary classes and instantiate the Google Gemini and OpenAI GPT models as specified in the instructions.



In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

## Define agent tools (optional but recommended)

### Subtask:
Although not explicitly requested, defining tools for each agent (e.g., knowledge base lookups, external API calls) will enhance their capabilities. This step will be marked as optional in the plan.


**Reasoning**:
Define placeholder functions for the tools that each agent might use.



## Define agent nodes

### Subtask:
Create a node for each agent (Charging Station, Booking , Cafe, General). Each node will contain the logic for that agent.


**Reasoning**:
Define the Python functions for each agent (Charging Station, Booking , Cafe, General) as described in the instructions, using the previously initialized language models.



In [7]:
## FULL CODE IN THIS BLOCK

from langgraph.graph import StateGraph
from typing import TypedDict, Annotated, Union
import operator
from langchain_core.prompts import ChatPromptTemplate

from langgraph.graph import StateGraph
from typing import TypedDict, Annotated, Union
import operator
from langchain_core.prompts import ChatPromptTemplate

# Define a state for the graph using TypedDict
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        query: Charging Station Query
        classification: classification of the query
        chargin_station_enquiry_response: str
        booking_response: str
        cafe_enquiry_response: str
        general_response: str
        final_response: str
    """
    query: str
    classification: str
    chargingStationEnquiry_response: str
    booking_response: str
    cafeEnquiry_response: str
    general_response: str
    final_response: str


#Redefine the classifier_agent to return a dictionary to update the state
def classifier_agent(state: dict) -> dict:
    """Classifies the customer query and updates the state."""
    query = state['query']
    print(f"--- Classifier Agent Start ---")
    print(f"Input Query: {query}")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that classifies customer queries into one of the following categories: booking, charging, cafe, general. Respond with only the category name."),
        ("human", "Classify the following query: {query}")
    ])
    chain = prompt | gemini_llm
    category = chain.invoke({"query": query}).content.strip().lower() # Ensure clean output
    # Basic parsing to extract the category
    if "charging" in category:
        classified_category = "charging"
    elif "booking" in category:
        classified_category = "booking"
    elif "cafe" in category:
        classified_category = "cafe"
    else:
        classified_category = "general"

    print(f"Classified as: {classified_category}")
    print(f"Output State Update: {{'classification': '{classified_category}'}}")
    print(f"--- Classifier Agent End ---")

    # Return a dictionary to update the state
    return {"classification": classified_category}


# Redefine the specialized agents to return dictionaries to update the state
def chargingStationEnquiry_agent(state: dict) -> dict:
    """Handles charging station availability queries between 2 locations and updates the state."""
    query = state['query']
    print(f"--- charging Agent Start ---")
    print(f"Input Query: {query}")
    prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are an intelligent EV charging assistant. "
        "Your task is to answer questions about the availability of all electric vehicle charging stations "
        "between two given locations. Be factual, concise, and user-friendly. "
        "If possible, mention key routes or highway corridors connecting the two cities."
    ),
    (
        "human",
        "Query: {query}\n\n"
        "Provide a short, clear response summarizing where charging stations are available "
        "along the route, and any useful travel tips for EV users."
    )
])
    chain = prompt | gemini_llm
    response = chain.invoke({"query": query}).content
    print(f"Response: {response}")
    print(f"Output State Update: {{'chargingStationEnquiry_response': '{response}'}}")
    print(f"--- Charging Agent End ---")
    return {"chargingStationEnquiry_response": response}


def booking_agent(state: dict) -> dict:
    return

    # Agent to find cafes nearby to the location shared
def cafeEnquiry_agent(state: dict) -> dict:
    """Consider the locality shared and suggest cafes."""
    query = state['query']
    print(f"--- Cafe Agent Start ---")
    print(f"Input Query: {query}")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that can suggest cafes based on location described in {query}."
        "Consdier only those cafes which have EV charging station available with it."
        "Consider only the cafes which are open and have review rating of 4 and above."
        ),
        ("human", "Respond to the following cafe related query: {query}")
    ])
    chain = prompt | gemini_llm
    response = chain.invoke({"query": query}).content
    print(f"Response: {response}")
    print(f"Output State Update: {{'cafe_enquiry_response': '{response}'}}")
    print(f"--- Cafe Agent End ---")
    return {"cafe_enquiry_response": response}

def general_agent(state: dict) -> dict:
    """Handles general queries and updates the state."""
    query = state['query']
    print(f"--- General Agent Start ---")
    print(f"Input Query: {query}")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that responds to general queries."),
        ("human", "Respond to the following general query: {query}")
    ])
    chain = prompt | gemini_llm
    response = chain.invoke({"query": query}).content
    print(f"Response: {response}")
    print(f"Output State Update: {{'general_response': '{response}'}}")
    print(f"--- General Agent End ---")
    return {"general_response": response}

# Redefine the router node
def router(state: dict) -> str:
    """Routes the query based on the classifier's output."""
    classification = state.get('classification')
    print(f"--- Router Start ---")
    print(f"Current state: {state}")
    print(f"Received classification: {classification}")

    if classification == "charging":
        next_node = "chargingStationEnquiry_agent"
    elif classification == "booking":
        next_node = "booking_agent"
    elif classification == "cafe":
        next_node = "cafeEnquiry_agent"
    else:
        # Fallback to general agent for any other classification
        next_node = "general_agent"

    print(f"Routing to: {next_node}")
    print(f"--- Router End ---")
    return next_node

# Redefine the aggregate_response node
def aggregate_response(state: dict) -> dict:
    """Collects responses from specialized agents and formats a single response."""
    print(f"--- Aggregation Node ---")
    print(f"Current state before aggregation: {state}")

    chargingStationEnquiry_response = state.get('chargingStationEnquiry_response', "")
    booking_response = state.get('booking_response', "")
    cafeEnquiry_response = state.get('cafeEnquiry_response', "")
    general_response = state.get('general_response', "")

    responses = []
    if chargingStationEnquiry_response:
        responses.append(f"charging Station Enquiry Response: {chargingStationEnquiry_response}")
    if booking_response:
        responses.append(f"booking Response: {booking_response}")
    if cafeEnquiry_response:
        responses.append(f"cafe Enquiry Response: {cafeEnquiry_response}")
    if general_response:
        responses.append(f"General Response: {general_response}")

    final_response = "\n".join(responses)
    # Update the state with the final response.
    # Ensure this is a dictionary update as expected by StateGraph.
    update_dict = {'final_response': final_response}

    print(f"Aggregated Final Response: {final_response}")
    print(f"Output State Update: {update_dict}")
    print(f"------------------------")
    return update_dict


# Re-compile the graph with the updated nodes
# Instantiate a StateGraph with the defined state
workflow = StateGraph(GraphState)

# Add the updated agent nodes and the router node
workflow.add_node("classifier_agent", classifier_agent)
workflow.add_node("chargingStationEnquiry_agent", chargingStationEnquiry_agent)
workflow.add_node("booking_agent", booking_agent)
workflow.add_node("cafeEnquiry_agent", cafeEnquiry_agent)
workflow.add_node("general_agent", general_agent)
workflow.add_node("router", router)
workflow.add_node("aggregate_response", aggregate_response)


# Set the entry point of the graph
workflow.set_entry_point("classifier_agent")
# workflow.add_edge("classifier_agent", "router")

# Instead, use add_conditional_edges from classifier_agent:
workflow.add_conditional_edges(
    "classifier_agent",  # source node
    router,              # routing function
    {
        "charging": "chargingStationEnquiry_agent",
        "booking": "booking_agent",
        "cafe": "cafeEnquiry_agent",
        "general": "general_agent"
    }
)

# REMOVE explicit edges from specialized agents to aggregate_response
# workflow.add_edge("billing_agent", "aggregate_response")
# workflow.add_edge("technical_agent", "aggregate_response")
# workflow.add_edge("general_agent", "aggregate_response")

# Set the aggregate_response node as the end point of the graph
# We will now add edges from the specialized agents to the aggregate_response node
workflow.add_edge("chargingStationEnquiry_agent", "aggregate_response")
workflow.add_edge("booking_agent", "aggregate_response")
workflow.add_edge("cafeEnquiry_agent", "aggregate_response")
workflow.add_edge("general_agent", "aggregate_response")

workflow.set_finish_point("aggregate_response")


# Compile the graph
app = workflow.compile()

# Define sample customer queries
sample_queries = [
    "I am travelling from Bangalore to Pune find a charging station on my way?",  # charging station  query
]

# Iterate through sample queries and test the workflow
for query in sample_queries:
    print(f"=== Running workflow for query: {query} ===")
    # Invoke the compiled LangGraph application
    # The initial state should contain the input query
    result = app.invoke({"query": query})
    # The final response is stored in the 'final_response' key of the final state
    print(f"--- Final Result ---")
    print(f"Input Query: {query}")
    print(f"Final Response: {result.get('final_response', 'No final response generated.')}")
    print(f"===========================================\n")

=== Running workflow for query: I am travelling from Bangalore to Pune find a charging station on my way? ===
--- Classifier Agent Start ---
Input Query: I am travelling from Bangalore to Pune find a charging station on my way?
Classified as: charging
Output State Update: {'classification': 'charging'}
--- Classifier Agent End ---
--- Router Start ---
Current state: {'classification': 'charging', 'query': 'I am travelling from Bangalore to Pune find a charging station on my way?'}
Received classification: charging
Routing to: chargingStationEnquiry_agent
--- Router End ---


KeyError: 'chargingStationEnquiry_agent'